# Capítulo 13 - Mini Projeto 4

In [1]:
# Importando bibliotecas:
from platform import python_version
print('Versão Python:', python_version(), '\n')

import findspark
findspark.init() # Inicializando o findspark

import pyspark
from pyspark import SparkContext # SparkContext é a conexão principal para criar RDDs
from pyspark.sql import SparkSession # SparkSession é a entrada principal para trabalhar com dados estruturados em Spark
from pyspark.sql import Window # Window é uma classe para criar janelas de dados
from pyspark.sql.functions import col # col é uma função para acessar uma coluna em um DataFrame
from pyspark.sql.functions import row_number # row_number é uma função para adicionar um número de linha a um DataFrame
from pyspark.sql.functions import lead # lead é uma função para acessar a próxima linha em um DataFrame
from pyspark.sql.functions import min, max # min e max são funções para encontrar o valor mínimo e máximo de uma coluna
from pyspark.sql.functions import unix_timestamp # unix_timestamp é uma função para converter uma string em um timestamp    

%reload_ext watermark
%watermark -a "gustavogzr" --iversions

Versão Python: 3.11.1 

Author: gustavogzr

pyspark  : 3.5.2
findspark: 2.0.1



## Preparando o Ambiente Spark

In [2]:
# Criar o Spark Context
sc = SparkContext(appName="Mini-Projeto4")
# Criar a Spark Session
spark = SparkSession.builder.getOrCreate()
spark # imprime a sessão criada

## Carregando os dados para DataFrame Spark

In [4]:
# Nome do arquivo:
arquivo = '.arquivos_DSA/dados/dataset.txt'
# Carregando os dados para um DataFrame Spark:
df = spark.read.csv(arquivo, header=True)
df.show(5)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+
only showing top 5 rows



In [5]:
type(df)

pyspark.sql.dataframe.DataFrame

## Criando tabela temporária para executar queries SQL

In [6]:
# Criar tabela temporária:
df.createOrReplaceTempView("tb_logistica")

## Executando queries SQL

In [7]:
# Verificando as colunas da tabela:
spark.sql("SHOW COLUMNS FROM tb_logistica").show()

+----------+
|  col_name|
+----------+
|id_veiculo|
|   entrega|
|   horario|
+----------+



In [8]:
# Visualizando os 5 primeiros registros:
spark.sql("SELECT * FROM tb_logistica LIMIT 5").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+



In [9]:
# Realizar describe da tabela:
spark.sql("DESCRIBE tb_logistica").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|id_veiculo|   string|   NULL|
|   entrega|   string|   NULL|
|   horario|   string|   NULL|
+----------+---------+-------+



## Queries SQL vs Dot Notation no Spark SQL

In [10]:
# Query SQL
spark.sql('SELECT id_veiculo AS veiculo, entrega FROM tb_logistica LIMIT 5').show()

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
+-------+---------+



In [12]:
# Dot Notation - vantagem: não precisa de tabelas temporárias
df.select(
    col('id_veiculo').alias('veiculo'),
    col('entrega')
).limit(5).show()

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
+-------+---------+



## Usando funções SQL no Spark SQL

In [13]:
df.columns # visualizando as colunas

['id_veiculo', 'entrega', 'horario']

In [14]:
# converter o dataframe Spark para um dataframe Pandas
pandasDF = df.toPandas()

In [15]:
type(pandasDF)

pandas.core.frame.DataFrame

In [16]:
pandasDF

,id_veiculo,entrega,horario
0,298,Entrega 1,7:58a
1,298,Entrega 2,8:04a
2,298,Entrega 3,8:17a
3,298,Entrega 4,8:28a
4,298,Entrega 5,8:33a
5,298,Entrega 6,8:39a
6,298,Entrega 7,9:07a
7,315,Entrega 1,6:05a
8,315,Entrega 2,6:14a
9,315,Entrega 3,6:24a


### Métodos Select e Collect

In [17]:
# Selecionando dados de duas colunas:
df.select('id_veiculo', 'entrega').show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [18]:
# Método alternativo:
df.select(df.id_veiculo, df.entrega).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [19]:
# A função col é outra alternativa:
df.select(col('id_veiculo'), col('entrega')).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [20]:
# Podemos selecionar todas as colunas presentes em uma lista:
lista_colunas = ['id_veiculo', 'entrega']
df.select(*lista_colunas).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [21]:
# Mesmo exemplo anterior, mas agora utilizando o list comprehension:
df.select([coluna for coluna in lista_colunas]).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [26]:
# Renomeando colunas para facilitar a manipulação:
df.select('id_veiculo', 'entrega').withColumnRenamed('id_veiculo', 'veiculo').show(10)

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
|    298|Entrega 6|
|    298|Entrega 7|
|    315|Entrega 1|
|    315|Entrega 2|
|    315|Entrega 3|
+-------+---------+
only showing top 10 rows



In [28]:
# Também é possível renomear colunas com a função alias:
df.select(
    col('id_veiculo').alias('veiculo'),
    'entrega'
).show(10)

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
|    298|Entrega 6|
|    298|Entrega 7|
|    315|Entrega 1|
|    315|Entrega 2|
|    315|Entrega 3|
+-------+---------+
only showing top 10 rows



In [24]:
# Selecionando colunas pelo índice
df.select(df.columns[2:]).show(3)

+-------+
|horario|
+-------+
|  7:58a|
|  8:04a|
|  8:17a|
+-------+
only showing top 3 rows



In [29]:
# Selecionando colunas através de expressões regulares:
df.select(df.colRegex("`^.*Entrega*`")).show()

+---------+
|  entrega|
+---------+
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
|Entrega 7|
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
|Entrega 7|
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
+---------+
only showing top 20 rows



In [30]:
# O Dataframe original não é alterado:
df.show(30)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
|       315|Entrega 5|  6:45a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 7|  7:32a|
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       457|Entrega 6|  6:21a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+

